In [ ]:
pip install pytrends matplotlib pandas seaborn plotly

In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import matplotlib.pyplot as plt	
import seaborn as sns
import plotly.express as px
from pytrends.request import TrendReq


# Setup Pytrends library and keywords

In [ ]:
pydrends = TrendReq(hl='en-US', tz=360)
keywords=" DATA SCIENCE"

# Data Request

In [ ]:
import time

# Increase sleep time and add retry logic
success = False
attempts = 0
while not success and attempts < 5:
	try:
		pydrends.build_payload([keywords], cat=0, timeframe='today 12-m', geo='', gprop='')
		success = True
	except Exception as e:
		print(f"Attempt {attempts+1}: {e}. Retrying after 120 seconds...")
		time.sleep(120)
		attempts += 1
if not success:
	raise Exception("Failed to build payload after multiple attempts due to TooManyRequestsError.")

# Country Wise Interest

In [ ]:
region_data=pydrends.interest_by_region()
region_data=region_data.sort_values(by=keywords, ascending=False).head(15)

In [ ]:
region_data = pydrends.interest_by_region()
region_data = region_data.sort_values(by=keywords, ascending=False).head(15)

plt.figure(figsize=(10,6))
sns.barplot(x=region_data[keywords], y=region_data.index, palette='Blues')
plt.title(f"Top Countries searching for '{keywords}' ")
plt.xlabel('Interest Level')
plt.ylabel('Countries')
plt.show()

# World Map

In [ ]:
region_data = region_data.reset_index()
region_data = region_data.rename(columns={'index': 'geoName'})
fig = px.choropleth(
	region_data,
	locations='geoName',
	locationmode='country names',
	color=keywords,
	title=f"Search Interest for '{keywords}' by Country",
	color_continuous_scale='Blues'
)
fig.show()

# Time Wise Interest

In [ ]:
import time

# Retry logic for interest_over_time
success = False
attempts = 0
while not success and attempts < 5:
	try:
		time_df = pydrends.interest_over_time()
		success = True
	except Exception as e:
		print(f"Attempt {attempts+1}: {e}. Retrying after 120 seconds...")
		time.sleep(120)
		attempts += 1
if not success:
	raise Exception("Failed to fetch time_df after multiple attempts due to ConnectionError.")

In [ ]:

if 'time_df' not in globals():
	time_df = pydrends.interest_over_time()

plt.Figure(figsize=(12,6))
plt.plot(time_df.index, time_df[keywords], marker='o', color='purple')
plt.title(f"Time Wise Interest over '{keywords}'")
plt.xlabel("Date")
plt.ylabel("Interest")
plt.grid(True)
plt.show()

# Multiple Keywords Comparing

In [ ]:
from pytrends.request import TrendReq

kw_list = ["cloud computing", "data science", "machine learning"]

# Ensure pydrends is defined
if 'pydrends' not in globals():
	pydrends = TrendReq(hl='en-US', tz=360)

import time

# Add retry logic to handle TooManyRequestsError
success = False
attempts = 0
while not success and attempts < 5:
	try:
		pydrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo='', gprop='')
		success = True
	except Exception as e:
		print(f"Attempt {attempts+1}: {e}. Retrying after 120 seconds...")
		time.sleep(120)
		attempts += 1
if not success:
	raise Exception("Failed to build payload after multiple attempts due to TooManyRequestsError.")

In [ ]:
import matplotlib.pyplot as plt

# Ensure kw_list is defined
kw_list = ["cloud computing", "data science", "machine learning"]

# Ensure pydrends is defined
if 'pydrends' not in globals():
	from pytrends.request import TrendReq
	pydrends = TrendReq(hl='en-US', tz=360)

# Build payload with kw_list before fetching interest_over_time
pydrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo='', gprop='')
compare_df = pydrends.interest_over_time()

# Remove 'isPartial' column if present
if 'isPartial' in compare_df.columns:
	compare_df = compare_df.drop(columns=['isPartial'])

# Check if all keywords are present in the DataFrame columns
missing_keywords = [kw for kw in kw_list if kw not in compare_df.columns]
if missing_keywords:
	raise KeyError(f"Missing keyword columns in DataFrame: {missing_keywords}")

plt.figure(figsize=(12,6))
for kw in kw_list:
	plt.plot(compare_df.index, compare_df[kw], marker='o', label=kw)
plt.title("Keywords Comparison Over Time")
plt.xlabel("Date")
plt.ylabel("Interest")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()